In [1]:
import pandas as pd
from npi.npi import NPI
from npi.utils.utils import longprint
from npi.pecos import PECOS, medicare_program_engagement, medical_school

### Instantiate an NPI object

The NPI class provides convenient access to the NPI data housed in our shared folders.

The canonical way to obtain an instance of this class is via (where entities=1 is specified if you only want to work with individuals)

In [2]:
npi = NPI(entities=1)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_entity
* Metadata: {'func': 'get_entity', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'get_entity': '-code snipped-'}} 
* (identified) Called functions: ['get_entity']
* Cache found; loaded from ID 1592587996398162
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



### Display a list of data that can be currently managed by adding as an attibute

In [3]:
npi.display_gettable_attributes()

Gettable raw data:
 ['PLICNUM', 'PLICSTATE', 'entity', 'npideactdate', 'npireactdate', 'pcredential', 'pcredentialoth', 'pfname', 'pfnameoth', 'pgender', 'plname', 'plnameoth', 'ploccityname', 'plocline1', 'plocline2', 'plocstatename', 'ploctel', 'ploczip', 'pmname', 'pmnameoth', 'pnamesuffix', 'pnamesuffixoth', 'ptaxcode'] 


Gettable processed data:
 ['credentials', 'expanded_fullnames', 'fullnames', 'licenses', 'practitioner_type', 'removaldate', 'secondary_practice_locations']


### Retrieve all the data you need 

Hint: if you want to get rid of all the bulky output, run the below cell twice. The second time will be instantaneous and will clear the output.

In [4]:
npi.retrieve('removaldate')
npi.retrieve('fullnames')
npi.retrieve('practitioner_type')
npi.retrieve('plocstatename')
npi.retrieve('credentials')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_deactdate
* Metadata: {'func': 'get_deactdate', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'get_deactdate': '-code snipped-'}} 
* (identified) Called functions: ['get_deactdate']
* Cache found; loaded from ID 1592590259085239
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_reactdate
* Metadata: {'func': 'get_reactdate', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'ge

### Construct a dataset of NPI removals that can be used to search for people

1. Get only doctors and advanced practice nurses for now

In [5]:
npi.practitioner_type.set_index('npi')[['MD/DO', 'CNM', 'CNS', 'CRNA', 'NP', 'Other APRN']]

,MD/DO,CNM,CNS,CRNA,NP,Other APRN
npi,,,,,,
1003000100,0,0,0,0,0,0
1003000126,1,0,0,0,0,0
1003000134,1,0,0,0,0,0
1003000142,1,0,0,0,0,0
1003000159,0,0,0,0,1,0
...,...,...,...,...,...,...
1992999833,0,0,0,0,0,0
1992999858,0,0,0,0,0,0
1992999866,0,0,0,0,0,0


In [6]:
s = npi.practitioner_type.set_index('npi')[['MD/DO', 'CNM', 'CNS', 'CRNA', 'NP', 'Other APRN']].sum(axis=1) > 0 
mds_aprns = s[s].reset_index().drop(columns=0)

2. Merge together the MD/APRN list with the deactivation list and their credentials

In [7]:
removed = npi.removaldate.merge(mds_aprns)
removed = removed.merge(npi.credentials)
removed = removed.drop(columns='pcredential_stripped')
removed

,npi,npideactdate,pcredential,cat
0,1003001660,2016-05-27,MD,MD/DO
1,1003003971,2019-11-04,MD,MD/DO
2,1003004326,2017-04-28,DO,MD/DO
3,1003006339,2014-04-30,MD,MD/DO
4,1003006479,2014-05-05,MD,MD/DO
...,...,...,...,...
51551,1992995732,2010-12-20,MD,MD/DO
51552,1992996169,2014-02-10,MD,MD/DO
51553,1992997316,2014-10-14,MD,MD/DO
51554,1992997761,2019-06-10,MD,MD/DO


3. Merge in the state locations in the month they are removed

Here we are selecting the last month observed for each NPI, for convenience

In [8]:
states = npi.plocstatename.groupby('npi', as_index=False).last().drop(columns='month')

In [9]:
removed = removed.merge(states)

4. Merge in full names

In [10]:
removed = removed.merge(npi.fullnames)

### 60 most recent removals

People can be duplicated if they have more than one value for fullname or credential. You can use this additional context to find people

In [11]:
removed.sort_values(['npideactdate', 'npi']).tail(60)

,npi,npideactdate,pcredential,cat,plocstatename,pfname,pmname,plname,pnamesuffix,othflag
39546,1710933239,2020-06-09,MD,MD/DO,CA,MAHNAZ,GHAFFARI,KASSAI,,1
39872,1720061708,2020-06-09,NP-C,NP,MA,PAUL,B,COUSINEAU,,0
40528,1730171034,2020-06-09,FNP,NP,TX,EMMALOU,,KEYES,,0
41414,1740555358,2020-06-09,MSN ANP-BC,NaN,LA,RICHARD,E,DAVIS,,0
41415,1740555358,2020-06-09,MSN,NaN,LA,RICHARD,E,DAVIS,,0
41416,1740555358,2020-06-09,APRN,Other APRN,LA,RICHARD,E,DAVIS,,0
41417,1740555358,2020-06-09,ANP-BC,NaN,LA,RICHARD,E,DAVIS,,0
41894,1750488318,2020-06-09,MD,MD/DO,OH,JOHN,T,MAKLEY,,0
42598,1770505406,2020-06-09,MD,MD/DO,MI,ROBYN,J,ARRINGTON,JR,0
43112,1770982688,2020-06-09,MD,MD/DO,CA,LAWRENCE,KUNUK,KIM,,0


### Finding deactivations that are unlikely to be retired already

This finds people with NPIs who are active in the Medicare data, and also pulls their medical school or training graduation date.

In [12]:
active = medicare_program_engagement() 
med_school = medical_school()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: physician_compare_select_vars
* Metadata: {'func': 'physician_compare_select_vars', 'args': [[]], 'kwargs': {'drop_duplicates': False, 'date_var': True}, 'code': {'physician_compare_select_vars': '-code snipped-', 'detect_date': '-code snipped-', 'convert_dtypes': '-code snipped-'}} 
* (identified) Called functions: ['physician_compare_select_vars', 'detect_date', 'convert_dtypes']
* Cache found; loaded from ID 1591669125237018
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
active

,npi,PC_Min_Year,PC_Max_Year,PartD_Min_Year,PartD_Max_Year,PartB_Min_Year,PartB_Max_Year,maybe_active,active_2020
0,1003000126,2013,2020,2013,2017,2012,2017,True,True
1,1003000134,2013,2020,<NA>,<NA>,2012,2017,True,True
2,1003000142,2013,2020,2013,2017,2012,2017,True,True
3,1003000159,2014,2015,2014,2014,<NA>,<NA>,False,False
4,1003000381,2013,2014,<NA>,<NA>,2012,2012,False,False
...,...,...,...,...,...,...,...,...,...
2065784,1992996672,<NA>,<NA>,<NA>,<NA>,2012,2017,True,False
2065785,1992996995,<NA>,<NA>,<NA>,<NA>,2012,2013,False,False
2065786,1992998157,<NA>,<NA>,<NA>,<NA>,2012,2015,False,False
2065787,1992998702,<NA>,<NA>,<NA>,<NA>,2012,2017,True,False


In [14]:
med_school

,npi,Medical school name,Graduation year
0,1003001363,UNIVERSITY OF ARKANSAS COLLEGE OF MEDICINE,2003
1,1003001843,OTHER,1998
2,1003001017,"UNIVERSITY OF CALIFORNIA, IRVINE, CALIFORNIA C...",1968
3,1235197823,OTHER,1980
4,1780720284,OTHER,1982
...,...,...,...
1567657,1548515877,OTHER,1997
1567658,1598747891,OTHER,1993
1567659,1629204938,OTHER,2005
1567660,1700104353,"UNIVERSITY OF CALIFORNIA, SAN DIEGO SCHOOL OF ...",2010


In [15]:
pecos = PECOS(['NPI', 
               'Medical school name', 
               'Graduation year', 
               'Organization legal name',
               'Group Practice PAC ID',
               'Number of Group Practice members',
               'Hospital affiliation CCN 1',
               'Hospital affiliation LBN 1',
               'Hospital affiliation CCN 2',
               'Hospital affiliation LBN 2',
               'Hospital affiliation CCN 3',
               'Hospital affiliation LBN 3',
               'Hospital affiliation CCN 4',
               'Hospital affiliation LBN 4',
               'Hospital affiliation CCN 5',
               'Hospital affiliation LBN 5'],
              drop_duplicates=False, date_var=True)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: physician_compare_select_vars
* Metadata: {'func': 'physician_compare_select_vars', 'args': [['NPI', 'Medical school name', 'Graduation year', 'Organization legal name', 'Group Practice PAC ID', 'Number of Group Practice members', 'Hospital affiliation CCN 1', 'Hospital affiliation LBN 1', 'Hospital affiliation CCN 2', 'Hospital affiliation LBN 2', 'Hospital affiliation CCN 3', 'Hospital affiliation LBN 3', 'Hospital affiliation CCN 4', 'Hospital affiliation LBN 4', 'Hospital affiliation CCN 5', 'Hospital affiliation LBN 5'], False, True], 'kwargs': {}, 'code': {'physician_compare_select_vars': '-code snipped-', 'detect_date': '-code snipped-', 'convert_dtypes': '-code snipped-'}} 
* (identified) Called functions: ['physician_compare_select_vars', 'detect_date', 'convert_dtype

In [16]:
pecos.physician_compare.sort_values(['NPI', 'date'])

,NPI,Medical school name,Graduation year,Organization legal name,Group Practice PAC ID,Number of Group Practice members,Hospital affiliation CCN 1,Hospital affiliation LBN 1,Hospital affiliation CCN 2,Hospital affiliation LBN 2,Hospital affiliation CCN 3,Hospital affiliation LBN 3,Hospital affiliation CCN 4,Hospital affiliation LBN 4,Hospital affiliation CCN 5,Hospital affiliation LBN 5,date
33024288,1003000126,OTHER,1994,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,7113900655,72,210027,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,511315,"POTOMAC VALLEY HOSPITAL OF W VA, INC",210017,GARRETT COUNTY MEMORIAL HOSPITAL,511309,"WAR MEMORIAL HOSPITAL, INC.",490005,"WINCHESTER MEDICAL CENTER, INC.",2013-06-27
33024289,1003000126,OTHER,1994,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,7113900655,72,210027,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,511315,"POTOMAC VALLEY HOSPITAL OF W VA, INC",210017,GARRETT COUNTY MEMORIAL HOSPITAL,511309,"WAR MEMORIAL HOSPITAL, INC.",490005,"WINCHESTER MEDICAL CENTER, INC.",2013-06-27
33024290,1003000126,OTHER,1994,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,7113900655,72,210027,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,511315,"POTOMAC VALLEY HOSPITAL OF W VA, INC",210017,GARRETT COUNTY MEMORIAL HOSPITAL,511309,"WAR MEMORIAL HOSPITAL, INC.",490005,"WINCHESTER MEDICAL CENTER, INC.",2013-06-27
33024291,1003000126,OTHER,1994,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,7113900655,72,210027,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,511315,"POTOMAC VALLEY HOSPITAL OF W VA, INC",210017,GARRETT COUNTY MEMORIAL HOSPITAL,511309,"WAR MEMORIAL HOSPITAL, INC.",490005,"WINCHESTER MEDICAL CENTER, INC.",2013-06-27
33024292,1003000126,OTHER,1994,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,7113900655,72,210027,WESTERN MARYLAND HEALTH SYSTEM CORPORATION,511315,"POTOMAC VALLEY HOSPITAL OF W VA, INC",210017,GARRETT COUNTY MEMORIAL HOSPITAL,511309,"WAR MEMORIAL HOSPITAL, INC.",490005,"WINCHESTER MEDICAL CENTER, INC.",2013-06-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4257511,1992999874,OTHER,2002,PRIMEDOC OF RICHMOND PC,6305833823,36,490069,BON SECOURS MEMORIAL REGIONAL MEDICAL CENTER,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,2017-10-01
64147836,1992999874,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,2002,PRIMEDOC OF RICHMOND PC,6305833823,33,490069,BON SECOURS MEMORIAL REGIONAL MEDICAL CENTER,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,2017-12-01
64147837,1992999874,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,2002,PRIMEDOC OF RICHMOND PC,6305833823,33,490069,BON SECOURS MEMORIAL REGIONAL MEDICAL CENTER,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,2017-12-01
57010535,1992999874,MEDICAL COLLEGE OF VIRGINIA COMMONWEALTH UNIVE...,2002,SOUTHEASTERN INTENSIVIST SERVICES PC,9335152107,135,490112,CJW MEDICAL CENTER,490069,BON SECOURS MEMORIAL REGIONAL MEDICAL CENTER,490118,HENRICO DOCTORS' HOSPITAL,490059,BON SECOURS ST MARYS HOSPITAL,NaN,<NA>,2020-04-02


In [19]:
removed.merge(med_school, how='left').merge(active, how='left').sort_values(['npideactdate', 'npi']).reset_index(drop=True)

,npi,npideactdate,pcredential,cat,plocstatename,pfname,pmname,plname,pnamesuffix,othflag,Medical school name,Graduation year,PC_Min_Year,PC_Max_Year,PartD_Min_Year,PartD_Max_Year,PartB_Min_Year,PartB_Max_Year,maybe_active,active_2020
0,1417160383,2007-12-07,MD,MD/DO,PR,JOSE,,LLORENS,JR,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
1,1164488441,2007-12-19,MD,MD/DO,KY,PORTER,,RAMSEY,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
2,1669549481,2008-01-03,MD,MD/DO,OH,KI,HWAN,LEE,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3,1043267164,2008-01-09,MD,MD/DO,CO,KENNETH,M,RAFAL,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,1255416913,2008-01-09,MD,MD/DO,CA,YUEH-MING,,WANG,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55304,1366771198,2020-06-11,MD,MD/DO,CA,ROBERT,,HERSCH,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
55305,1447351465,2020-06-11,MD,MD/DO,NJ,THOMAS,FRANCIS,BEJGROWICZ,,0,<NA>,<NA>,<NA>,<NA>,2017,2017,<NA>,<NA>,True,False
55306,1376745570,2020-06-12,MD,MD/DO,CA,DALE,WILLIAM,ZEH,,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
55307,1952308553,2020-06-12,MD,MD/DO,CA,ROLLAND,C,ZIEHL,,0,GEORGE WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,1957,2013,2015,<NA>,<NA>,2012,2014,False,False
